# Worklist
## EDA: 
correlation, each feature count plot, label plot, outliers? label transformation? 1-9 to low medium high
split:4898 -> 
## Feature
keep

## Modeling

** SVM **

** Random Forest **

** Naive Bayes **

** DT **

** KNN=1 **

## Evaluation

Confusion Matrix


# Combine Each One's Work Here

In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import collections
from sklearn.preprocessing import LabelEncoder
import imblearn
from collections import Counter
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks,ClusterCentroids
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# wine.shape
# wine.head()
# wine.columns
# wine.isnull().sum()
# # no missing values
# wine.dtypes
# # all features are quantitive

# DATA

In [ ]:
# This is white wine
wine = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv',delimiter=";")

In [ ]:
quality = wine["quality"].values
category = []
for num in quality:
    if num<5:
        category.append("Low")
    elif num>6:
        category.append("High")
    else:
        category.append("Midium")
category = pd.DataFrame(data=category, columns=["category"])
data = pd.concat([wine,category],axis=1)
data.drop(columns="quality",axis=1,inplace=True)
X = data.iloc[:,:-1].values
y = data.iloc[:,-1].values

In [ ]:
collections.Counter(y)

In [ ]:
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=2018)

In [ ]:
collections.Counter(y)

In [ ]:
################### resample ##################
# under sample "2" ； over sample 1,0
smt = ClusterCentroids(ratio = {2:1500})
X_sm,y_sm = smt.fit_sample(X_train,y_train)
smt2 =SMOTE(ratio = {0:1000,1:1500})
X_sm2,y_sm2 = smt2.fit_sample(X_sm,y_sm)
Counter(y_sm2)

## **Reagan**

grid search  --> "best" parameters  
model ori data with "best" params  
model resampled data with "best" params  
visualize two confusion matrices

In [ ]:
# strtifiedKFold
skf = StratifiedKFold(n_splits=4)
for training, validation in skf.split(X_train, y_train):
    print("%s %s" % (len(training),len(validation)))

kernel=precomputed : X should be a square kernel matrix  
kernel=poly: poly requires another para: degree.  
kernel=linear: work but pretty slow

In [ ]:
# grid search
pipe_svm=Pipeline([('clf', svm.SVC())])   
grid_params = dict(clf__C=[0.1, 0.3, 1, 3, 10],
                   clf__gamma=[0.1, 0.3, 1, 3, 10],
                   clf__kernel=['rbf','sigmoid'])  
gs = GridSearchCV(estimator=pipe_svm,  
                  param_grid=grid_params,
                  scoring='accuracy',
                  cv=skf)
gs.fit(X_train, y_train)

In [ ]:
gs.best_params_

In [ ]:
# model before resample
opt=svm.SVC(kernel='rbf', C=1, gamma=3)   # the best model
opt.fit(X_train,y_train)  # fit the model
predicted = opt.predict(X_test)
print(classification_report(y_test, predicted))
print("---------------------------------------------------------")
print("The accuracy score of SVM is %s" % accuracy_score(y_test, predicted))

In [ ]:
# model after resample
opt=svm.SVC(kernel='rbf', C=1, gamma=3)   # the best model
opt.fit(X_sm2,y_sm2)  # fit the model
sm_predicted = opt.predict(X_test)
print(classification_report(y_test, predicted))
print("---------------------------------------------------------")
print("The accuracy score of SVM (resampled) is %s" % accuracy_score(y_test, predicted))

In [ ]:
# relabel back : 0 means good, 1 for low, 2 for medium 
# better visualization for confusion matrix
y_test_re = list(y_test)
for i in range(len(y_test_re)):
    if y_test_re[i] == 0:
        y_test_re[i] = "good"
    if y_test_re[i] == 1:
        y_test_re[i] = "low"
    if y_test_re[i] == 2:
        y_test_re[i] = "medium"
pred_re = list(predicted)
for i in range(len(predicted)):
    if pred_re[i] == 0:
        pred_re[i] = "good"
    if pred_re[i] == 1:
        pred_re[i] = "low"
    if pred_re[i] == 2:
        pred_re[i] = "medium"
sm_pred_re = list(sm_predicted)
for i in range(len(sm_predicted)):
    if sm_pred_re[i] == 0:
        sm_pred_re[i] = "good"
    if sm_pred_re[i] == 1:
        sm_pred_re[i] = "low"
    if sm_pred_re[i] == 2:
        sm_pred_re[i] = "medium"
svm_confusion = pd.crosstab(pd.Series(y_test_re,name='actual'), pd.Series(pred_re,name='predicted'))
sm_svm_confusion = pd.crosstab(pd.Series(y_test_re,name='actual'), pd.Series(sm_pred_re,name='predicted'))

In [ ]:
svm_confusion

In [ ]:
sm_svm_confusion

##  **Luke**

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%matplotlib inline
# remove warnings
import warnings
warnings.simplefilter("ignore")

### Import Data 

In [ ]:
# This is white wine
wine = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv',delimiter=";")
# This is red wine
datared = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv',delimiter=";")
print("Done!")

We might only focus on ** white wine** for this project.

### Some basic EDA 

In [ ]:
wine.columns

#### Bar Plot of Response 

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(wine["quality"],palette="muted")
wine["quality"].value_counts()

The distribution is fairly normal, but small freq for high and low score

#### Data Transformation 

We want to transfer the score(num) to low-medium-high quality level(categorical) by: 

3,4 -> low

56 -> medium

789 -> high

In [ ]:
quality = wine["quality"].values
category = []
for num in quality:
    if num<5:
        category.append("Low")
    elif num>6:
        category.append("High")
    else:
        category.append("Midium")
print("Done!")

In [ ]:
[(i,category.count(i)) for i in set(category)]

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(category,palette="muted")

#### Create our new data frame  

In [ ]:
category = pd.DataFrame(data=category, columns=["category"])
# Cby concatenationoncatenation
data = pd.concat([wine,category],axis=1)
# Drop the score
data.drop(columns="quality",axis=1,inplace=True)

In [ ]:
data.head(3)

#### Cor Mat to check features 

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(wine.corr(),annot=True)

Lokks ok to me, only Density~Residual Sugar shows a large correlation (0.84). We might remove one of them for modeling if necessary  in the future

5 features: citric acid and sugar, free sulfur dioxide, ph, sulphates shows a low correlation.

### Set up model matrix

In [ ]:
X = data.iloc[:,:-1].values
y = data.iloc[:,-1].values

In [ ]:
X.shape,y.shape

In [ ]:
# may need pca for better resampling result
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
pca = PCA(n_components=11)
X = pca.fit_transform(X)
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

In [ ]:
import collections
collections.Counter(y)

0 means good, 1 for low, 2 for medium

### Train & Test split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=2018)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [ ]:
X_train.shape,X_test.shape

In [ ]:
X_train.shape,y_train.shape

We have 3918 training data, 980 test. Will use test for final model accuracy comparision. Only use training set to train our model.

#### 1.Random Forest

**the following is rfc without resampling**

In [ ]:
X_train.shape,y_train.shape

In [ ]:
rfc = RandomForestClassifier(n_estimators=250,random_state = 2018)
# might need CV for parameter tuning here
rfc.fit(X_train, y_train)
pred_rfc = rfc.predict(X_test)
print(classification_report(y_test, pred_rfc))
print("The RF model accuracy is %s" % accuracy_score(y_test, pred_rfc))

**confusion matrix**

In [ ]:
# relabel back : 0 means good, 1 for low, 2 for medium for better visualization
y_test_re = list(y_test)
for i in range(len(y_test_re)):
  if y_test_re[i] == 0:
    y_test_re[i] = "good"
  if y_test_re[i] == 1:
    y_test_re[i] = "low"
  if y_test_re[i] == 2:
    y_test_re[i] = "medium"
pred_rfc_re = list(pred_rfc)
for i in range(len(pred_rfc_re)):
  if pred_rfc_re[i] == 0:
    pred_rfc_re[i] = "good"
  if pred_rfc_re[i] == 1:
    pred_rfc_re[i] = "low"
  if pred_rfc_re[i] == 2:
    pred_rfc_re[i] = "medium"

In [ ]:
y_actu = pd.Series(y_test_re, name='Actual')
y_pred = pd.Series(pred_rfc_re, name='Predicted')
rfc_confusion = pd.crosstab(y_actu, y_pred)

In [ ]:
rfc_confusion

#### resampling here

In [ ]:
import imblearn
from collections import Counter
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks,ClusterCentroids

In [ ]:
# under sample "2"
smt = ClusterCentroids(ratio = {2:1200})
X_sm,y_sm = smt.fit_sample(X_train,y_train)
Counter(y_sm)

In [ ]:
# over sample 1,0
smt2 =SMOTE(ratio = {0:1200,1:1200})
X_sm2,y_sm2 = smt2.fit_sample(X_sm,y_sm)
Counter(y_sm2)

In [ ]:
X_sm2.shape,y_sm2.shape

In [ ]:
y_sm.dtype
####!!!!! need to encoding again, fuck

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder_y2 = LabelEncoder()
y_train_sm2 = labelencoder_y2.fit_transform(y_sm2)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X2 = StandardScaler()
X_sm2 = sc_X2.fit_transform(X_sm2)
X_test = sc_X2.transform(X_test)

In [ ]:
rfc = RandomForestClassifier(n_estimators=150,random_state = 2018)
# might need CV for parameter tuning here
rfc.fit(X_sm2, y_train_sm2)
pred_rfc = rfc.predict(X_test)
print(classification_report(y_test, pred_rfc))
print("The RF model accuracy is %s" % accuracy_score(y_test, pred_rfc))

In [ ]:
# relabel back : 0 means good, 1 for low, 2 for medium for better visualization
y_test_re = list(y_test)
for i in range(len(y_test_re)):
  if y_test_re[i] == 0:
    y_test_re[i] = "good"
  if y_test_re[i] == 1:
    y_test_re[i] = "low"
  if y_test_re[i] == 2:
    y_test_re[i] = "medium"
pred_rfc_re = list(pred_rfc)
for i in range(len(pred_rfc_re)):
  if pred_rfc_re[i] == 0:
    pred_rfc_re[i] = "good"
  if pred_rfc_re[i] == 1:
    pred_rfc_re[i] = "low"
  if pred_rfc_re[i] == 2:
    pred_rfc_re[i] = "medium"

In [ ]:
y_actu = pd.Series(y_test_re, name='Actual')
y_pred = pd.Series(pred_rfc_re, name='Predicted')
rfc_confusion = pd.crosstab(y_actu, y_pred)

In [ ]:
rfc_confusion

We sacrificed 5% overall accuracy in change of increase 26% in low group and 15% in high group.